# Epsilon Constraint Method

### Setup

In [1]:
import os, sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cobra')

from cobra import io, Model, Solution
from scripts.helpers.model import rxn_in_model, add_single_gene_reaction_pair
from scripts.opt._fba import flux_balance_analysis
from scripts.opt._fva import run_flux_variability_analysis

import os, numpy as np, pandas as pd

BIOMASS = "Biomass_Chlamy_auto"
ERGEXCH = "ERGOSTEROLEXCH"
CO2_EX  = "EX_co2_e"

ERG = "ergosterol_c"

In [2]:
# Load models

# Load wildtype from manual directory (adjust path for notebooks directory)
# Use io.read_sbml_model for local files instead of io.load_model
wildtype = io.read_sbml_model('../data/fill/xmls/MNL_iCre1355_auto_GAPFILL.xml')

models = {
    "Wildtype": wildtype
}

# Get altered reactions of wildtype - using correct directory name
altered_dir = '../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/'

for root, dirs, files in os.walk(altered_dir):
    for file in files:
        if file.endswith('.xml'):
            # Get file name and remove .xml extension
            model_name = file[:-4]
            full_path = os.path.join(root, file)
            print(f"Loading model: {model_name} from {full_path}")
            models[model_name] = io.read_sbml_model(full_path)

for name in models.keys():
    model = models[name]

    # Add export reaction
    
    if not rxn_in_model(model, ERGEXCH):
        add_single_gene_reaction_pair(
            model=model, 
            gene_id="EXCHERG_GENE",
            reaction_id=ERGEXCH,
            reaction_name="Ergosterol exchange (assumption)", 
            reaction_subsystem="Exchange", 
            metabolites=[(-1, ERG)],
            reversible=True
        )


No objective coefficients in model. Unclear what should be optimized


Loading model: SQE+MVA from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQE+MVA.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQE from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQE.xml


No objective coefficients in model. Unclear what should be optimized


...

No objective coefficients in model. Unclear what should be optimized


Loading model: SQS+SQE from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/h\SQS+SQE.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQS from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/h\SQS.xml


No objective coefficients in model. Unclear what should be optimized


In [3]:
CO2_CAP = 10.0   # i tested it out. 10 is max (i.e. 80.0 has same results as 10.0)

# Set constraints on CO2 Exchange
def set_co2_cap(m, cap):
    r = m.reactions.get_by_id(CO2_EX)
    r.lower_bound = -float(cap)
    return r.lower_bound, r.upper_bound

# Epsilon constraint method
def epsilon_front(m, n=20):
    m = m.copy()
    pts = [] # of shape b, e, c

    with m:
        sb = flux_balance_analysis(m, objectives=[BIOMASS], is_pfba=True)
        if sb.status != "optimal":
            return np.zeros((0,3))
        bmax = float(sb.fluxes.get(BIOMASS, 0.0))

    for eps in np.linspace(0, bmax, n):
        with m:
            biom = m.reactions.get_by_id(BIOMASS)
            biom.lower_bound = max(biom.lower_bound, eps)
            m.objective = m.reactions.get_by_id(ERGEXCH)
            se = flux_balance_analysis(m, objectives=[ERGEXCH], is_pfba=True)

            b = float(se.fluxes.get(BIOMASS, 0.0))
            e = float(se.fluxes.get(ERGEXCH, 0.0))
            c = float(se.fluxes.get(CO2_EX, 0.0)) if rxn_in_model(m, CO2_EX) else np.nan
            pts.append((b, e, c))

    return np.array(pts)

In [4]:
all_fronts = []
summary_rows = []

for name in models.keys():
    base = models[name].copy()
    lb, ub = set_co2_cap(base, CO2_CAP)

    pts = epsilon_front(base, n=41)

    df = pd.DataFrame(pts, columns=["biomass","erg","co2"])
    df["variant"] = name
    df["CO2_cap"] = CO2_CAP
    df["ERG_per_CO2"] = df["erg"] / (df["co2"].abs().replace(0, np.nan))
    all_fronts.append(df)

    bmax = df["biomass"].max()
    near_zero = df.iloc[(df["biomass"]).abs().argsort()[:1]].iloc[0]

    summary_rows.append({
        "variant": name,
        "bmax": bmax,
        "erg_at_b0": float(near_zero["erg"]),
        "yield_at_b0": float(near_zero["ERG_per_CO2"]),
    })

fronts = pd.concat(all_fronts, ignore_index=True)
summary = pd.DataFrame(summary_rows).sort_values(["erg_at_b0","yield_at_b0","bmax"], ascending=False)

# saves
outdir = "../results/variant_compare_eps"
os.makedirs(outdir, exist_ok=True)
fronts.to_csv(os.path.join(outdir, "fronts_all_variants.csv"), index=False)
summary.to_csv(os.path.join(outdir, "summary_metrics.csv"), index=False)
print(f"Saved CSVs to: {outdir}")

Saved CSVs to: ../results/variant_compare_eps


In [6]:
import plotly.graph_objects as go

fig1 = go.Figure()
for name in summary["variant"]:
    sub = fronts[fronts["variant"] == name].sort_values("biomass")
    fig1.add_trace(go.Scatter(
        x=sub["biomass"], y=sub["erg"], mode="lines+markers", name=name,
    ))
fig1.update_layout(
    title=f"ERG vs Biomass at CO₂ cap {CO2_CAP}",
    xaxis_title="Biomass flux", yaxis_title="ERG export flux",
    width=900, height=600
)
fig1.write_html(os.path.join(outdir, "fronts_ERG_vs_Biomass.html"))

fig1